# Import packages

In [1]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder

# Load datasets -- training and validation

In [2]:
#Loading the training and validation datasets
#df_train = pd.read_csv("C:\HuiLi_work\SAS_work\Data\cs_accepts_train.csv")
#df_validation = pd.read_csv("C:\HuiLi_work\SAS_work\Data\cs_accepts_validation.csv")
df_train = pd.read_csv("C:\demodata\cs_accepts_train.csv")
df_validation = pd.read_csv("c:\demodata\cs_accepts_validation.csv")

In [ ]:
df_train.head()

# Conver the categorical to numeric variables

In [ ]:

# Importing LabelEncoder and initializing it
le=LabelEncoder()
#Iterating over all the common columns in train and test
for col in df_train.columns.values:
    #Encoding only categorical variables
    if df_train[col].dtypes=='object':
        #Using whole data to form an exhausitve list of levels
        df=df_validation[col].append(df_train[col])
        le.fit(df.values)
        df_train[col]=le.transform(df_train[col])
        df_validation[col]=le.transform(df_validation[col])

In [ ]:
df_train.head()

# List variable list

In [ ]:
numList = ["Nchildren","Nhousehold","Age","TimeAddress","TimeJob","tel","NumMyLoan","NumFinLoan","NumLoans",
          "Income","EC_Card","IncLevel","bureau","region","regionLarge","cash"]  ## with missing
                                                      ## without missing
#numList = ["Nchildren","Nhousehold","Age","tel","NumMyLoan",
#                "NumFinLoan","NumLoans","Income","EC_Card","IncLevel","bureau","region","regionLarge", 
#                "cash"] ## without missing    
catList =  ["title","status","resid","nat", "prof","car","card","saving"] 

# Convert the dataframes to arrays

In [ ]:
feat        =  numList+catList;
labels      = ['target']
trainFeat   = df_train.as_matrix(feat)              # training features
trainLabels = df_train.as_matrix(labels)            # training target
validationFeat   = df_validation.as_matrix(feat)    # validation features
validationLabels = df_validation.as_matrix(labels)  # validation target

# Check the missing data and impute the missing value with mean

In [ ]:
# Checking the number of missing for each coloumn
for i in range(0,len(feat)):
    aa =np.count_nonzero(~np.isnan(trainFeat[:,i]))
    print(aa)

In [ ]:
# Imputing the missing data
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
trainFeat_im = imp.fit_transform(trainFeat)
valiFeat_im = imp.fit_transform(validationFeat)

In [ ]:
for i in range(0,len(feat)):
    aa =np.count_nonzero(~np.isnan(valiFeat_im[:,i]))
    print aa

# Train random forest model 

In [ ]:
# Create the randome forest objuect which will include all the parameters for the fit
rf = RandomForestClassifier(n_estimators=10)
# Fit the training features to the training labels and build the random forest model 
rf.fit(trainFeat_im, trainLabels) 

# Test Validation dataset

In [ ]:
# Predict the scores of validation datasets
testPreds =rf.predict(valiFeat_im)
#print testPreds
metrics.confusion_matrix(validationLabels,testPreds)
metrics.accuracy_score(validationLabels,testPreds)

In [ ]:
#Compute the roc and auc for validation
validationPreds_proba =rf.predict_proba(valiFeat_im)[:,1]
fpr, tpr, tresh = metrics.roc_curve(validationLabels,validationPreds_proba)
auc = metrics.auc(fpr,tpr)
print auc

# Test training dataset

In [ ]:
# Predict the scores of train datasets
trainPreds =rf.predict(trainFeat_im)
#print testPreds
metrics.confusion_matrix(trainLabels,trainPreds)
metrics.accuracy_score(trainLabels,trainPreds)

In [ ]:
#Compute the roc and auc for training
trainPreds_proba =rf.predict_proba(trainFeat_im)[:,1]
fpr_train, tpr_train, tresh_train = metrics.roc_curve(trainLabels,trainPreds_proba)
auc_train = metrics.auc(fpr_train,tpr_train)
print auc_train